In [1]:
from keras.models import Sequential, load_model
from keras.layers import Dense, CuDNNLSTM, LSTM, Lambda
from keras import optimizers
from keras import callbacks
import numpy as np
from keras_tqdm import TQDMNotebookCallback
from sklearn.utils import shuffle
from tqdm import tqdm_notebook

Using TensorFlow backend.


In [2]:
# code used liberally from https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py

sonnets = []
with open("../data/shakespeare.txt") as f:
    line = f.readline()
    while line:
        # Flag start of sonnet, read in next 14 lines
        if any(char.isdigit() for char in line):
            curr_sonnet = ""
            for i in range(14):
                curr_sonnet += f.readline().strip().lower()
                curr_sonnet += "\n" if i != 13 else ""
            sonnets.append(curr_sonnet)
        line = f.readline()
        
# Vectorization prep
chars = sorted(list(set("".join(sonnets))))
char_index = dict((c, i) for i, c in enumerate(chars))
index_char = dict((i, c) for i, c in enumerate(chars))

# Read subsequences from each sonnet, add to training list
# Don't read across sonnets?
length = 40
step = 1
tr_data = []
tar_char = []
for s in sonnets:
    for i in range(0, len(s) - length, step):
        tr_data.append(s[i:i+length])
        tar_char.append(s[i+length])
        
tr_data_full = []
tar_char_full = []
sonnets_full = "\n".join(sonnets)
for i in range(0, len(sonnets_full) - length, step):
    tr_data_full.append(sonnets_full[i:i+length])
    tar_char_full.append(sonnets_full[i+length])
    
# Vectorize training data
X = np.zeros((len(tr_data), length, len(chars)), dtype=np.bool)
Y = np.zeros((len(tr_data), len(chars)), dtype=np.bool)

X_full = np.zeros((len(tr_data_full), length, len(chars)), dtype=np.bool)
Y_full = np.zeros((len(tr_data_full), len(chars)), dtype=np.bool)

for i, seq in enumerate(tr_data):
    for j, char in enumerate(seq):
        X[i, j, char_index[char]] = 1
    Y[i, char_index[tar_char[i]]] = 1
    
for i, seq in enumerate(tr_data_full):
    for j, char in enumerate(seq):
        X_full[i, j, char_index[char]] = 1
    Y_full[i, char_index[tar_char_full[i]]] = 1
    
X_full_shuff, Y_full_shuff = shuffle(X_full, Y_full)

In [4]:
model = Sequential()
model.add(LSTM(128, input_shape=(length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               85504     
_________________________________________________________________
dense_1 (Dense)              (None, 38)                4902      
Total params: 90,406
Trainable params: 90,406
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(X, Y, epochs=10, validation_split=0.1, batch_size=128)

Train on 78584 samples, validate on 8732 samples
Epoch 1/10
78584/78584 [==============================] - 36s 463us/step - loss: 2.1044 - val_loss: 2.0401
Epoch 2/10
78584/78584 [==============================] - 36s 455us/step - loss: 1.9595 - val_loss: 1.9569
Epoch 3/10
78584/78584 [==============================] - 35s 446us/step - loss: 1.8694 - val_loss: 1.8935
Epoch 4/10
78584/78584 [==============================] - 35s 443us/step - loss: 1.8000 - val_loss: 1.8544
Epoch 5/10
78584/78584 [==============================] - 34s 439us/step - loss: 1.7452 - val_loss: 1.8207
Epoch 6/10
78584/78584 [==============================] - 35s 447us/step - loss: 1.7002 - val_loss: 1.7980
Epoch 7/10
78584/78584 [==============================] - 36s 453us/step - loss: 1.6588 - val_loss: 1.7753
Epoch 8/10
78584/78584 [==============================] - 35s 442us/step - loss: 1.6247 - val_loss: 1.7579
Epoch 9/10
78584/78584 [==============================] - 35s 439us/step - loss: 1.5923 - val_l

In [7]:
amodel = Sequential()
amodel.add(LSTM(128, input_shape=(length, len(chars))))
amodel.add(Dense(len(chars), activation='softmax'))
amodel.compile(loss="categorical_crossentropy", optimizer="adam")
amodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               85504     
_________________________________________________________________
dense_2 (Dense)              (None, 38)                4902      
Total params: 90,406
Trainable params: 90,406
Non-trainable params: 0
_________________________________________________________________


In [8]:
amodel.fit(X, Y, epochs=10, validation_split=0.1, batch_size=128)

Train on 78584 samples, validate on 8732 samples
Epoch 1/10
78584/78584 [==============================] - 36s 455us/step - loss: 2.6588 - val_loss: 2.2856
Epoch 2/10
78584/78584 [==============================] - 35s 449us/step - loss: 2.1919 - val_loss: 2.1182
Epoch 3/10
78584/78584 [==============================] - 35s 440us/step - loss: 2.0526 - val_loss: 2.0236
Epoch 4/10
78584/78584 [==============================] - 34s 438us/step - loss: 1.9471 - val_loss: 1.9341
Epoch 5/10
78584/78584 [==============================] - 35s 448us/step - loss: 1.8607 - val_loss: 1.8848
Epoch 6/10
78584/78584 [==============================] - 35s 443us/step - loss: 1.8021 - val_loss: 1.8509
Epoch 7/10
78584/78584 [==============================] - 35s 446us/step - loss: 1.7567 - val_loss: 1.8143
Epoch 8/10
78584/78584 [==============================] - 34s 439us/step - loss: 1.7151 - val_loss: 1.7918
Epoch 9/10
78584/78584 [==============================] - 35s 445us/step - loss: 1.6817 - val_l

In [9]:
namodel = Sequential()
namodel.add(LSTM(128, input_shape=(length, len(chars))))
namodel.add(Dense(len(chars), activation='softmax'))
namodel.compile(loss="categorical_crossentropy", optimizer="nadam")
namodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               85504     
_________________________________________________________________
dense_3 (Dense)              (None, 38)                4902      
Total params: 90,406
Trainable params: 90,406
Non-trainable params: 0
_________________________________________________________________


In [10]:
namodel.fit(X, Y, epochs=10, validation_split=0.1, batch_size=128)

Train on 78584 samples, validate on 8732 samples
Epoch 1/10
78584/78584 [==============================] - 36s 453us/step - loss: 2.4465 - val_loss: 2.1392
Epoch 2/10
78584/78584 [==============================] - 35s 448us/step - loss: 2.0254 - val_loss: 1.9742
Epoch 3/10
78584/78584 [==============================] - 35s 448us/step - loss: 1.8815 - val_loss: 1.8643
Epoch 4/10
78584/78584 [==============================] - 35s 449us/step - loss: 1.7878 - val_loss: 1.8172
Epoch 5/10
78584/78584 [==============================] - 35s 445us/step - loss: 1.7117 - val_loss: 1.7897
Epoch 6/10
78584/78584 [==============================] - 35s 441us/step - loss: 1.6620 - val_loss: 1.7498
Epoch 7/10
78584/78584 [==============================] - 36s 461us/step - loss: 1.6208 - val_loss: 1.7397
Epoch 8/10
78584/78584 [==============================] - 35s 444us/step - loss: 1.5828 - val_loss: 1.7214
Epoch 9/10
78584/78584 [==============================] - 35s 450us/step - loss: 1.5500 - val_l

In [17]:
bmodel = Sequential()
bmodel.add(CuDNNLSTM(128, input_shape=(length, len(chars))))
bmodel.add(Dense(len(chars), activation='softmax'))
bmodel.compile(loss="categorical_crossentropy", optimizer="rmsprop")
bmodel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 128)               86016     
_________________________________________________________________
dense_5 (Dense)              (None, 38)                4902      
Total params: 90,918
Trainable params: 90,918
Non-trainable params: 0
_________________________________________________________________


In [18]:
bmodel.fit(X, Y, epochs=10, validation_split=0.1, batch_size=128)

Train on 78584 samples, validate on 8732 samples
Epoch 1/10
78584/78584 [==============================] - 9s 114us/step - loss: 2.6415 - val_loss: 2.3185
Epoch 2/10
78584/78584 [==============================] - 8s 96us/step - loss: 2.2153 - val_loss: 2.1285
Epoch 3/10
78584/78584 [==============================] - 8s 97us/step - loss: 2.0599 - val_loss: 2.0311
Epoch 4/10
78584/78584 [==============================] - 8s 96us/step - loss: 1.9599 - val_loss: 1.9648
Epoch 5/10
78584/78584 [==============================] - 8s 96us/step - loss: 1.8878 - val_loss: 1.9188
Epoch 6/10
78584/78584 [==============================] - 7s 93us/step - loss: 1.8288 - val_loss: 1.8824
Epoch 7/10
78584/78584 [==============================] - 7s 92us/step - loss: 1.7811 - val_loss: 1.8456
Epoch 8/10
78584/78584 [==============================] - 7s 93us/step - loss: 1.7386 - val_loss: 1.8346
Epoch 9/10
78584/78584 [==============================] - 7s 92us/step - loss: 1.7019 - val_loss: 1.8137
Epoch

In [ ]:
bmodel.fit(X, Y, epochs=100, validation_split=0.1, batch_size=128, initial_epoch=10)

Train on 78584 samples, validate on 8732 samples
Epoch 11/100
78584/78584 [==============================] - 8s 96us/step - loss: 1.6374 - val_loss: 1.7826
Epoch 12/100
78584/78584 [==============================] - 7s 93us/step - loss: 1.6082 - val_loss: 1.7711
Epoch 13/100
78584/78584 [==============================] - 7s 92us/step - loss: 1.5820 - val_loss: 1.7568
Epoch 14/100
78584/78584 [==============================] - 7s 92us/step - loss: 1.5565 - val_loss: 1.7596
Epoch 15/100
78584/78584 [==============================] - 7s 92us/step - loss: 1.5322 - val_loss: 1.7594
Epoch 16/100
78584/78584 [==============================] - 7s 92us/step - loss: 1.5087 - val_loss: 1.7599
Epoch 17/100
78584/78584 [==============================] - 7s 92us/step - loss: 1.4861 - val_loss: 1.7566
Epoch 18/100
78584/78584 [==============================] - 7s 94us/step - loss: 1.4644 - val_loss: 1.7670
Epoch 19/100
78584/78584 [==============================] - 7s 92us/step - loss: 1.4426 - val_l

In [25]:
bmodel.fit(X, Y, epochs=200, validation_split=0.1, batch_size=128, initial_epoch=100)

Train on 78584 samples, validate on 8732 samples
Epoch 101/200
78584/78584 [==============================] - 7s 92us/step - loss: 0.6705 - val_loss: 2.9944
Epoch 102/200
78584/78584 [==============================] - 7s 89us/step - loss: 0.6682 - val_loss: 2.9917
Epoch 103/200
78584/78584 [==============================] - 7s 89us/step - loss: 0.6678 - val_loss: 3.0015
Epoch 104/200
78584/78584 [==============================] - 7s 89us/step - loss: 0.6671 - val_loss: 3.0111
Epoch 105/200
78584/78584 [==============================] - 7s 90us/step - loss: 0.6615 - val_loss: 3.0075
Epoch 106/200
78584/78584 [==============================] - 7s 89us/step - loss: 0.6592 - val_loss: 3.0302
Epoch 107/200
78584/78584 [==============================] - 7s 91us/step - loss: 0.6612 - val_loss: 3.0431
Epoch 108/200
78584/78584 [==============================] - 7s 89us/step - loss: 0.6549 - val_loss: 3.0351
Epoch 109/200
78584/78584 [==============================] - 7s 89us/step - loss: 0.655

In [ ]:
bmodel.fit(X, Y, epochs=250, validation_split=0.1, batch_size=256, initial_epoch=200)

In [ ]:
model2 = Sequential()
model2.add(CuDNNLSTM(256, input_shape=(length, len(chars)), return_sequences=True))
model2.add(CuDNNLSTM(256))
model2.add(Lambda(lambda x: x / 1.))
model2.add(Dense(len(chars), activation='softmax'))
opt2 = optimizers.RMSprop(lr=0.0002)
model2.compile(loss="categorical_crossentropy", optimizer=opt2, metrics=["accuracy"])
model2.summary()
history2 = model2.fit(X_full_shuff, Y_full_shuff,
                  epochs=80,
                  validation_split=0.05,
                  callbacks=[callbacks.ModelCheckpoint('../models/alexcdot_cudnnlstm_2_256_layer_rms_0002_nostep_best_val_t_1.h5', save_best_only=True),
                             callbacks.ModelCheckpoint('../models/alexcdot_cudnnlstm_2_256_layer_rms_0002_nostep_best_loss_t_1.h5', monitor="loss", save_best_only=True)])
model2.save('../models/alexcdot_cudnnlstm_2_512_layer_rms_0002_nostep_40_epochs_lr_t_1.h5')

print("Done T=1.0")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_3 (CuDNNLSTM)     (None, 40, 256)           303104    
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 256)               526336    
_________________________________________________________________
lambda_2 (Lambda)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 38)                9766      
Total params: 839,206
Trainable params: 839,206
Non-trainable params: 0
_________________________________________________________________
Train on 88909 samples, validate on 4680 samples
Epoch 1/80
88909/88909 [==============================] - 67s 757us/step - loss: 2.6870 - acc: 0.2453 - val_loss: 2.3504 - val_acc: 0.3327
Epoch 2/80
88909/88909 [==============================] - 67s 751us/step - loss:

In [2]:
main_seed = "shall i compare thee to a summer's day?\n"

def sample(m, seed):
    # print(seed)
    x_pred = np.zeros((1, length, len(chars)))
    for t, char in enumerate(seed):
        x_pred[0, t, char_index[char]] = 1
    probs = m.predict(x_pred)[0]
    return np.random.choice(np.arange(len(probs)), p=probs)

def create_sonnet(seed, n_lines, m):
    sonnet = ""
    curr_seed = seed
    while n_lines >= 0:
        next_ind = sample(m, curr_seed)
        next_char = index_char[next_ind]
        curr_seed = seed[1:] + next_char
        sonnet += next_char
        if next_char == "\n":
            n_lines -= 1
            print(n_lines)
    return sonnet

def create_sonnet_fixed_lines(seed, n_lines, m):
    sonnet = []
    curr_seed = seed
    for i in tqdm_notebook(range(n_lines)):
        curr_line = ""
        for j in range(len(seed)-1):
            next_ind = sample(m, curr_seed)
            next_char = index_char[next_ind]
            # while next_char == "\n":
            #    print(probs)
            #    next_ind = sample(m, curr_seed)
            #    next_char = index_char[next_ind]
            curr_seed = curr_seed[1:] + next_char
            curr_line += next_char
        # Artifically induce next line
        curr_seed = curr_seed[1:] + "\n"
        sonnet.append(curr_line)
    return sonnet


def create_sonnet_no_lines(seed, n_lines, m):
    sonnet = seed
    curr_seed = seed
    for i in range((n_lines-1) * len(seed)):
        next_ind = sample(m, curr_seed)
        next_char = index_char[next_ind]
        curr_seed = seed[1:] + next_char
        sonnet += next_char
    return sonnet

In [3]:
main_seed = "shall i compare thee to a summer's day?\n"
print(create_sonnet(main_seed, 14, 5))
# print(create_sonnet_no_lines(main_seed, 14, temperature=5.0))

NameError: name 'np' is not defined

In [1]:
1

1